In [ ]:
using System;
using System.Net;
using System.Net.Sockets;
using System.Text;
using System.Threading;

// Define and start the server
TcpListener server = new TcpListener(IPAddress.Any, 5000);
server.Start();
Console.WriteLine("Server started on port 5000. Waiting for clients...");

void StartServer()
{
    Thread serverThread = new Thread(() =>
    {
        while (true)
        {
            var client = server.AcceptTcpClient();
            Console.WriteLine("Client connected!");

            Thread clientThread = new Thread(HandleClient);
            clientThread.Start(client);
        }
    });

    serverThread.IsBackground = true;
    serverThread.Start();
}

void HandleClient(object clientObject)
{
    TcpClient client = (TcpClient)clientObject;
    NetworkStream stream = client.GetStream();

    byte[] buffer = new byte[1024];
    while (true)
    {
        try
        {
            int bytesRead = stream.Read(buffer, 0, buffer.Length);
            if (bytesRead == 0) break;

            string receivedMessage = Encoding.UTF8.GetString(buffer, 0, bytesRead);
            Console.WriteLine("Received: " + receivedMessage);

            string response = "Score received: " + receivedMessage;
            byte[] responseBytes = Encoding.UTF8.GetBytes(response);
            stream.Write(responseBytes, 0, responseBytes.Length);
        }
        catch (Exception ex)
        {
            Console.WriteLine("Error: " + ex.Message);
            break;
        }
    }

    client.Close();
    Console.WriteLine("Client disconnected.");
}

// Start the server
StartServer();


Server started on port 5000. Waiting for clients...


In [ ]:
using System;
using System.Net.Sockets;
using System.Text;

// Connect to the server and send scores
TcpClient client = new TcpClient();
client.Connect("127.0.0.1", 5000);
Console.WriteLine("Connected to server.");

NetworkStream stream = client.GetStream();

void SendScore(string score)
{
    byte[] data = Encoding.UTF8.GetBytes(score);
    stream.Write(data, 0, data.Length);

    byte[] buffer = new byte[1024];
    int bytesRead = stream.Read(buffer, 0, buffer.Length);
    string response = Encoding.UTF8.GetString(buffer, 0, bytesRead);

    Console.WriteLine("Server Response: " + response);
}

// Send a test score
SendScore("100");

// Clean up
stream.Close();
client.Close();
Console.WriteLine("Disconnected from server.");


Connected to server.
Server Response: Score received: 100
Disconnected from server.
